Before running the file Upload all your data set on your goole drive in a zip format

In [1]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#before running this please change the RUNTIME to GPU (Runtime -> Change runtime type -> set harware accelarotor as GPU)
#download and unzip the data from google drive Colab environment
from google_drive_downloader import GoogleDriveDownloader as gdd
#use only file id of the link
#Note: Below link is just an example, Not an actual link. Actual Links are in ReadMe file
#https://drive.google.com/file/d/1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07/view?usp=sharing
url = '1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07'
gdd.download_file_from_google_drive(file_id = url,dest_path='./data.zip',unzip=True)

In [2]:
#To get the average frame count
import json
import glob
import numpy as np
import cv2
import copy
#change the path accordingly
video_files =  glob.glob('/content/drive/MyDrive/Dataset/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [379, 398, 349, 314, 339, 308, 376, 327, 364, 284, 309, 316, 224, 315, 334, 390, 435, 308, 337, 345, 311, 459, 363, 325, 305, 466, 352, 452, 332, 303, 400, 489, 459, 510, 410, 272, 317, 340, 315, 333, 247, 312, 269, 306, 298, 274, 201, 376, 364, 398, 339, 489, 316, 510, 452, 340, 400, 352, 327, 317, 459, 309, 308, 314, 349, 363, 435, 466, 284, 315, 325, 272, 410, 308, 345, 224, 311, 303, 315, 334, 247, 312, 306, 201, 305, 298, 274, 409, 272, 276, 365, 371, 380, 380, 505, 409, 307, 328, 505, 322, 376, 376, 425, 383, 383, 307, 425, 322, 161, 458, 458, 161, 270, 270, 215, 469, 303, 350, 529, 326, 459, 534, 479, 464, 520, 469, 303, 350, 529, 459, 534, 479, 520, 239, 351, 517, 318, 172, 502, 535, 322, 343, 517, 318, 161, 496, 535, 373, 441, 357, 313, 350, 327, 316, 320, 371, 397, 330, 349, 441, 357, 313, 337, 327, 316, 320, 371, 397, 330, 400, 451, 422, 519, 412, 400, 451, 422, 412, 445, 361, 380, 470, 516, 445, 361, 380, 470, 516, 380, 428, 372, 499, 398, 409, 361, 458, 497, 380, 42

In [3]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image
!pip3 install face_recognition
!mkdir '/content/drive/My Drive/Face_Only_Data'
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=d0fabd7fda511abac427a0ae6fa8d01cfc85eeaca9ad2b344db94c90f3274edd
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


<ipython-input-3-d280777aa0c5>:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
create_face_videos(video_files,'/content/drive/My Drive/Face_Only_Data/')

No of videos already present  0


  0%|          | 0/292 [00:00<?, ?it/s]